In [20]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import tensorflow as tf
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt

In [21]:
train_images1 = sorted(os.listdir('../input/chest-xray-pneumonia/chest_xray/train/NORMAL'))
train_images2 = sorted(os.listdir('../input/chest-xray-pneumonia/chest_xray/train/PNEUMONIA'))
train_images = train_images1+train_images2
print(len(train_images))
del train_images1, train_images2

In [3]:
train_image = []
for im in train_images:
    try:
        img = image.load_img('../input/chest-xray-pneumonia/chest_xray/train/NORMAL/'+ im, target_size=(224,224), color_mode= 'grayscale')
    except:
        img = image.load_img('../input/chest-xray-pneumonia/chest_xray/train/PNEUMONIA/'+ im, target_size=(224,224), color_mode= 'grayscale')
    img = image.img_to_array(img)
    img = img/255
    train_image.append(img)
train_df = np.array(train_image)

In [4]:
import matplotlib.pyplot as plt

def show_img(dataset):
    f, ax = plt.subplots(1,5)
    f.set_size_inches(40, 20)
    for i in range(5,10):
        ax[i-5].imshow(dataset[i].reshape(224,224), cmap='gray')
    plt.show()

In [5]:
def add_noice(image):
    row,col,ch= image.shape
    mean = 0
    sigma = 1
    gauss = np.random.normal(mean,sigma,(row,col,ch))
    gauss = gauss.reshape(row,col,ch)
    noisy = image + gauss*0.07
    return noisy

In [6]:
noised_df= []

for img in train_df:
    noisy = add_noice(img)
    noised_df.append(noisy)
noised_df= np.array(noised_df)

In [7]:
show_img(train_df)

In [8]:
show_img(noised_df)

In [9]:
train_df.shape

In [10]:
xnoised = noised_df[0:4001]
xtrain = train_df[0:4001]
x_noisy_test= noised_df[4001:]
xtest = train_df[4001:]

In [11]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D,MaxPool2D ,UpSampling2D, Flatten, Input
from tensorflow.keras.optimizers import SGD, Adam, Adadelta, Adagrad
from tensorflow.keras import backend as K
from tensorflow.keras import layers, losses

def autoencoder():
    
    input_img = Input(shape=(224,224,1), name='image_input')
    
    #enoder 
    x = Conv2D(224, (3,3), activation='relu', padding='same', name='Conv1')(input_img)
    x = MaxPooling2D((2,2), padding='same', name='pool1')(x)
    x = Conv2D(224, (3,3), activation='relu', padding='same', name='Conv2')(x)
    x = MaxPooling2D((2,2), padding='same', name='pool2')(x)
    
    #decoder
    x = Conv2D(224, (3,3), activation='relu', padding='same', name='Conv3')(x)
    x = UpSampling2D((2,2), name='upsample1')(x)
    x = Conv2D(224, (3,3), activation='relu', padding='same', name='Conv4')(x)
    x = UpSampling2D((2,2), name='upsample2')(x)
    x = Conv2D(1, (3,3), activation='sigmoid', padding='same', name='Conv5')(x)
    
    #model
    autoencoder = Model(inputs=input_img, outputs=x)
    autoencoder.compile(optimizer='adam', loss=losses.MeanSquaredError())
    
    return autoencoder

In [12]:
model= autoencoder()
tf.keras.utils.plot_model(
    model, to_file='model.png'
)

In [13]:
from tensorflow.keras.callbacks import EarlyStopping

In [14]:
with tf.device('/device:GPU:0'):
    early_stopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=1, mode='auto')
    model.fit(xnoised, xtrain, epochs=100, batch_size=32, validation_data=(x_noisy_test, xtest), callbacks=[early_stopping])
    df=df.append({'Step':step,'Loss':loss.item()},ignore_index=True)
    df.plot(x='Step',y='Loss')


In [19]:
import cv2

pred = model.predict(x_noisy_test[:6])
def plot_predictions(y_true, y_pred):    
    f, ax = plt.subplots(3, 6)
    f.set_size_inches(15,12)
    for i in range(6):
        ax[0][i].set_title('Original Image '+str(i))
        ax[0][i].imshow(np.reshape(xtest[i], (224,224)), aspect='auto', cmap='gray')
        ax[1][i].set_title('Noise Added Image '+str(i))
        ax[1][i].imshow(np.reshape(y_true[i], (224,224)), aspect='auto', cmap='gray')
        ax[2][i].set_title('Denoised Image '+str(i))
        ax[2][i].imshow(np.reshape(y_pred[i], (224,224)), aspect='auto', cmap='gray')
       
    plt.tight_layout()
plot_predictions(x_noisy_test[:6], pred[:6])